**Punto 1**

In [4]:
import pandas as pd
import numpy as np
import nltk
import re
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, make_scorer, confusion_matrix
import string
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [5]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
df = pd.read_csv('/content/spam_dataset.csv', usecols =['text', 'label_num'])

In [7]:
df.head()

,text,label_num
0,Subject: enron methanol ; meter # : 988291\nth...,0
1,"Subject: hpl nom for january 9 , 2001\n( see a...",0
2,"Subject: neon retreat\nho ho ho , we ' re arou...",0
3,"Subject: photoshop , windows , office . cheap ...",1
4,Subject: re : indian springs\nthis deal is to ...,0


In [8]:
df.shape

(5171, 2)

In [9]:
df.isna().sum()

text         0
label_num    0
dtype: int64

Non ci sono valori mancanti nel dataframe.

In [10]:
df['label_num'].value_counts()

0    3672
1    1499
Name: label_num, dtype: int64

Il dataset è sbilanciato in favore della classe 0, quella delle mail non spam. In questo caso, per valutare il modello dobbiamo quindi controllare non solo la accuracy ma anche la recall perchè può essere che il modello tenda a classificare ogni osservazione come appartenente alla classe 0 risultando in una accuracy comunque alta. Guardando anche la recall, si controlla il numero di falsi negativi.

**Preprocessing del testo**

Guardando la colonna text del dataframe si intuisce che è presente spesso la parola "Subject", ovvero "oggetto" della mail. L'output della cella seguente indica che questa parola è presente in tutte le osservazioni della colonna text, non essendo informativa, si può omettere in fase di pulizia del testo.

In [11]:
df['text'].str.contains('Subject').all()

True

In [12]:
english_stopwords = stopwords.words('english')
nlp = spacy.load('en_core_web_sm')
punctuation = set(string.punctuation)


def data_cleaner(dataset):
    dataset_to_return = []
    for sentence in dataset:
        sentence = sentence.lower()
        sentence = sentence.replace("Subject", " ")
        for c in string.punctuation:
            sentence = sentence.replace(c, " ")
        document = nlp(sentence)
        sentence = ' '.join(token.lemma_ for token in document)
        sentence = ' '.join(word for word in sentence.split() if word not in english_stopwords)
        sentence = re.sub('\d', '', sentence)
        dataset_to_return.append(sentence)

    return dataset_to_return

In [13]:
text_cleaned = data_cleaner(df['text'])

**Costruzione del modello**

In [14]:
def bow_tfidf(dataset, tfidf_vectorizer):
    if tfidf_vectorizer == None:
        tfidf_vectorizer = TfidfVectorizer()
        X = tfidf_vectorizer.fit_transform(dataset)
    else:
        X = tfidf_vectorizer.transform(dataset)

    return X.toarray(), tfidf_vectorizer

In [15]:
Y = df['label_num'].values
X_train, X_test, Y_train, Y_test = train_test_split(text_cleaned, Y, test_size = 0.2, random_state = 43)

X_train, tfidf_fitted = bow_tfidf(X_train, None)
X_test, _ = bow_tfidf(X_test, tfidf_fitted)

In [16]:
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(activation='logistic',
                    hidden_layer_sizes=(100,),
                    max_iter=100,
                    solver='adam',
                    tol=0.005,
                    verbose=True)

clf.fit(X_train, Y_train)

Iteration 1, loss = 0.69312540
Iteration 2, loss = 0.56392712
Iteration 3, loss = 0.53150745
Iteration 4, loss = 0.50053331
Iteration 5, loss = 0.46735239
Iteration 6, loss = 0.43086553
Iteration 7, loss = 0.39105636
Iteration 8, loss = 0.34905743
Iteration 9, loss = 0.30727910
Iteration 10, loss = 0.26752570
Iteration 11, loss = 0.23128725
Iteration 12, loss = 0.19995000
Iteration 13, loss = 0.17307296
Iteration 14, loss = 0.15070748
Iteration 15, loss = 0.13198120
Iteration 16, loss = 0.11639859
Iteration 17, loss = 0.10341611
Iteration 18, loss = 0.09249709
Iteration 19, loss = 0.08321932
Iteration 20, loss = 0.07539059
Iteration 21, loss = 0.06865623
Iteration 22, loss = 0.06288217
Iteration 23, loss = 0.05787118
Iteration 24, loss = 0.05352343
Iteration 25, loss = 0.04973931
Iteration 26, loss = 0.04637356
Iteration 27, loss = 0.04340946
Iteration 28, loss = 0.04079467
Iteration 29, loss = 0.03845832
Iteration 30, loss = 0.03635298
Iteration 31, loss = 0.03449943
Iteration 32, los

MLPClassifier(activation='logistic', max_iter=100, tol=0.005, verbose=True)

In [17]:
def evaluate_model(y_true, y_pred):
    print(f"Recall: {recall_score(y_true, y_pred):.3f}")
    print(f"Accuracy: {accuracy_score(y_true, y_pred):.3f}")
    print(f"Precision: {precision_score(y_true, y_pred):.3f}")
    print(f"f1 score: {f1_score(y_true, y_pred):.3f}")
    print(f"MATRICE DI CONFUSIONE: \n {confusion_matrix(y_true, y_pred)}")

In [18]:
evaluate_model(Y_test, clf.predict(X_test))

Recall: 0.967
Accuracy: 0.986
Precision: 0.983
f1 score: 0.975
MATRICE DI CONFUSIONE: 
 [[725   5]
 [ 10 295]]


Vediamo se il modello presenta overfitting tramite la cross-validation.

In [19]:
from sklearn.model_selection import cross_val_score, KFold

kf = KFold(n_splits=5, shuffle =True)

Creo di nuovo il modello clf per omettere le stampe sulle varie epoche

In [20]:
clf = MLPClassifier(activation='logistic',
                    hidden_layer_sizes=(100,),
                    max_iter=100,
                    solver='adam',
                    tol=0.005,
                    verbose=False)

In [26]:
Y = df['label_num'].values
X = np.array(text_cleaned)

i=1
for train_index , test_index in kf.split(X):

    X_train , X_test = X[train_index] , X[test_index]
    Y_train, Y_test = Y[train_index], Y[test_index]


    X_train, tfidf_fitted = bow_tfidf(X_train, None)
    X_test, _ = bow_tfidf(X_test, tfidf_fitted)

    clf.fit(X_train, Y_train)

    print(f"Metriche dopo {i} split di cross-validation: \n")

    print(f"Metriche sul train:")
    evaluate_model(Y_train, clf.predict(X_train))


    print(f"Metriche sul test:")
    evaluate_model(Y_test, clf.predict(X_test))

    i+=1

Metriche dopo 1 split di cross-validation: 

Metriche sul train:
Recall: 0.999
Accuracy: 1.000
Precision: 0.999
f1 score: 0.999
MATRICE DI CONFUSIONE: 
 [[2944    1]
 [   1 1190]]
Metriche sul test:
Recall: 0.990
Accuracy: 0.991
Precision: 0.981
f1 score: 0.985
MATRICE DI CONFUSIONE: 
 [[721   6]
 [  3 305]]
Metriche dopo 2 split di cross-validation: 

Metriche sul train:
Recall: 1.000
Accuracy: 1.000
Precision: 1.000
f1 score: 1.000
MATRICE DI CONFUSIONE: 
 [[2932    0]
 [   0 1205]]
Metriche sul test:
Recall: 0.980
Accuracy: 0.985
Precision: 0.970
f1 score: 0.975
MATRICE DI CONFUSIONE: 
 [[731   9]
 [  6 288]]
Metriche dopo 3 split di cross-validation: 

Metriche sul train:
Recall: 0.999
Accuracy: 1.000
Precision: 0.999
f1 score: 0.999
MATRICE DI CONFUSIONE: 
 [[2945    1]
 [   1 1190]]
Metriche sul test:
Recall: 0.971
Accuracy: 0.988
Precision: 0.990
f1 score: 0.980
MATRICE DI CONFUSIONE: 
 [[723   3]
 [  9 299]]
Metriche dopo 4 split di cross-validation: 

Metriche sul train:
Recal

Come si può vedere dall'output della cella precedente, non c'è overfitting e il modello ha ottime metriche di accuracy e recall.

**Punto 2**

In [27]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from pprint import pprint
from gensim.models import CoherenceModel
from gensim.models.ldamodel import LdaModel
from gensim.corpora import Dictionary
from gensim import similarities

Selezioniamo le mail di spam:

In [28]:
df_spam = df[df['label_num'] == 1]

In [29]:
df_spam.shape

(1499, 2)

Aggiungiamo un passaggio in più al preprocessing: si può provare ad individuare le parole più frequenti in questo contesto che non sono informative. \
Per individuare le parole poco informative si può scegliere questa strada (simile all'idea che c'è dietro alla metrica tf-idf): attraverso il metodo "CountVectorizer" si ottiene una matrice in cui sulle colonne c'è il vocabolario e sulle righe le frasi del dataset. Dunque al posto (i,j) di questa matrice ci sarà il numero di volte che la parola j compare nel documento i. Si riportano tutte le entrate maggiori strettamente di 1 ad 1 (questo perchè non ci interessa quanto frequentemente compare quella parola all'interno del documento) e poi si fa la somma sulle colonne. Con questo approccio si riesce a vedere quali sono le parole che si ripetono più spesso nei vari documenti, che potrebbero essere poco significative.

In [30]:
spam_cleaned = data_cleaner(df_spam['text'])

count_vectorizer = CountVectorizer()

counts = count_vectorizer.fit_transform(spam_cleaned)
counts[counts > 1] = 1

word_counts = np.sum(counts, axis=0)
word_counts = np.asarray(word_counts).reshape(-1)

vocabulary = count_vectorizer.get_feature_names_out()

word_count_dict = {}
word_count_dict = dict(zip(vocabulary, word_counts))

sorted_word_count = {}
sorted_word_count = {k: v for k, v in sorted(word_count_dict.items(), key=lambda item: item[1], reverse=True)}

A questo punto andiamo a vedere le prime entrate del dizionario "sorted_word_count" creato nella cella precedente per eliminare delle parole che non sono significative. Scelgo di verificare manualmente (cioè con un approccio qualitativo) quali sono le parole da eliminare perchè se dovessi ad esempio fissare una soglia e tagliare le parole la cui frequenza è al di sopra di quella soglia rischierei di tagliare delle parole utili.

In [31]:
sorted_word_count

{'subject': 1499,
 'get': 476,
 'http': 475,
 'com': 444,
 'good': 316,
 'please': 314,
 'email': 296,
 'time': 296,
 'price': 295,
 'one': 280,
 'www': 263,
 'new': 253,
 'offer': 252,
 'need': 250,
 'use': 234,
 'go': 231,
 'want': 225,
 'take': 221,
 'information': 220,
 'click': 216,
 'free': 212,
 'like': 208,
 'product': 208,
 'make': 203,
 'send': 203,
 'look': 190,
 'work': 190,
 'money': 187,
 'online': 185,
 'message': 180,
 'see': 180,
 'know': 178,
 'today': 178,
 'may': 177,
 'order': 177,
 'also': 170,
 'would': 170,
 'thank': 167,
 'net': 163,
 'receive': 160,
 'day': 158,
 'include': 157,
 'available': 154,
 'stop': 154,
 'well': 154,
 'long': 152,
 'mail': 152,
 'remove': 152,
 'special': 151,
 'link': 150,
 'low': 150,
 'high': 148,
 'company': 146,
 'give': 146,
 'many': 143,
 'visit': 141,
 'save': 140,
 'find': 139,
 'service': 134,
 'without': 134,
 'year': 134,
 'list': 132,
 'site': 132,
 'pay': 129,
 'provide': 129,
 'result': 129,
 'dollar': 128,
 'back': 127,

Tra le parole che compaiono più spesso possiamo raggruppare in una lista quelle che sembrano meno informative.

In [32]:
non_informative = ['subject','get','http','com','please','good','email','time', 'one','www','need','new','go','take','want','click','like','make','look','thank',
                    'message','may','also','would','see','know','receive']

A questo punto possiamo usare il preprocessing fornito da gensim e poi allargare la classe delle stopwords con la lista "non_informative" e rimuovere stopwords e parole con lunghezza minore o uguale a 2. Quest'ultima cosa è dovuta al fatto che la parola "com" compariva molto spesso e quindi intuitivamente potrebbero esserci dei messaggi di spam che rimandano a siti web e quindi possiamo trovare dei "it", "de", "en" e cosi via.

In [33]:
english_stopwords.extend(non_informative)

def sent_to_words(dataset):
    for sentence in dataset:
        yield gensim.utils.simple_preprocess(str(sentence), deacc=True)

def remove_stopwords(dataset):
  for sentence in dataset:
    yield [word for word in sentence.split() if word not in english_stopwords and len(word)>2]

In [70]:
data_words = list(sent_to_words(list(remove_stopwords(spam_cleaned))))

In [71]:
id2word = corpora.Dictionary(data_words)

corpus = [id2word.doc2bow(text) for text in data_words]

Ora, il numero di topics è un parametro di cui fare il tuning, così come il parametro "passes"; non avendo problemi dal punto di vista computazionale visto che il dataset non è molto grande possiamo impostare passes a 10 e proviamo con 3 topics.

In [72]:
num_topics = 3

lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics,
                                       passes = 10,
                                       random_state=30 )
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.005*"price" + 0.004*"contact" + 0.004*"free" + 0.004*"computron" + '
  '0.003*"send" + 0.003*"remove" + 0.003*"mail" + 0.003*"account" + '
  '0.003*"money" + 0.003*"offer"'),
 (1,
  '0.007*"pill" + 0.004*"viagra" + 0.003*"cialis" + 0.003*"prescription" + '
  '0.003*"online" + 0.002*"drug" + 0.002*"med" + 0.002*"order" + 0.002*"soft" '
  '+ 0.002*"save"'),
 (2,
  '0.012*"company" + 0.007*"font" + 0.007*"statement" + 0.006*"stock" + '
  '0.006*"nbsp" + 0.005*"height" + 0.005*"information" + 0.005*"security" + '
  '0.004*"price" + 0.004*"width"')]


Già con num_topics=3 si ottiene un buon livello di coerenza.

Per fare tuning del parametro num_topics scegliamo un insieme di candidati e andiamo a valutare per ogni candidato la coherence (che misura appunto la coerenza delle parole all'interno di un topic) del modello LDA ottenuto.

In [73]:
def select_best_model(list_num_topics):

    text = list(sent_to_words(list(remove_stopwords(data_cleaner(df_spam['text'])))))
    coherence = []
    models = []

    for num_topics in list_num_topics:
        lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                              id2word=id2word,
                                              num_topics=num_topics,
                                              passes = 10,
                                              random_state = 50)
        models.append(lda_model)

        coherence_model_lda = CoherenceModel(model=lda_model, texts=text, dictionary=id2word, coherence='c_v')
        coherence.append(coherence_model_lda.get_coherence())

    max_index = coherence.index(max(coherence))
    selected_model = models[max_index]

    pprint(selected_model.print_topics())
    print(max(coherence))
    return selected_model, coherence

In [74]:
model, coherence = select_best_model([3,4,5,6,7,8,9,10])

[(0,
  '0.011*"font" + 0.009*"computron" + 0.007*"contact" + 0.007*"height" + '
  '0.006*"remove" + 0.006*"size" + 0.005*"free" + 0.005*"price" + 0.005*"line" '
  '+ 0.005*"list"'),
 (1,
  '0.007*"account" + 0.005*"money" + 0.004*"number" + 0.004*"international" + '
  '0.004*"claim" + 0.003*"business" + 0.003*"bank" + 0.003*"security" + '
  '0.003*"call" + 0.003*"mail"'),
 (2,
  '0.015*"nbsp" + 0.008*"height" + 0.007*"width" + 0.005*"border" + '
  '0.004*"soft" + 0.004*"back" + 0.004*"cialis" + 0.004*"well" + 0.004*"find" '
  '+ 0.004*"rate"'),
 (3,
  '0.017*"company" + 0.010*"statement" + 0.009*"stock" + 0.007*"information" + '
  '0.006*"security" + 0.006*"report" + 0.006*"investment" + 0.005*"within" + '
  '0.005*"price" + 0.004*"inc"'),
 (4,
  '0.012*"price" + 0.008*"window" + 0.008*"adobe" + 0.006*"software" + '
  '0.006*"professional" + 0.005*"office" + 0.005*"save" + 0.004*"microsoft" + '
  '0.004*"retail" + 0.004*"photoshop"'),
 (5,
  '0.002*"dosage" + 0.002*"die" + 0.001*"think

**Punto 3**

Per calcolare la distanza semantica tra i topics individuati al punto precedente, l'idea è quella di prendere un rappresentante per ogni topic e poi calcolare la cosine similarity tra i rappresentanti di ogni topic. \
Fissato un topic: \
1) scegliamo le parole più rilevanti (più probabili) associate a quel topic; \
2) calcoliamo la rappresentazione Word2Vec delle parole scelte al punto precedente; \
3) calcoliamo un vettore di media (elemento per elemento) dei vettori ottenuti al punto precedente (questo sarà il rappresentante del topic fissato). \
Iteriamo poi i 3 punti per ogni topic.

In [39]:
from gensim.models import Word2Vec
import gensim.downloader
from scipy import spatial

In [40]:
glove_vectors = gensim.downloader.load('glove-wiki-gigaword-300')

[==================================================] 100.0% 376.1/376.1MB downloaded


In [41]:
topic_representatives = []
for topic_id in range(model.num_topics):
    topic_words = [word for word, _ in model.show_topic(topic_id)]
    topic_embeddings = [glove_vectors[word] for word in topic_words if word in glove_vectors]
    if topic_embeddings:
        topic_centroid = sum(topic_embeddings) / len(topic_embeddings)
        topic_representatives.append(topic_centroid)
    else:
        topic_representatives.append(None)

A questo punto abbiamo i rappresentanti per ogni topic, nella cella seguente andiamo a calcolare le distanze tra i topic.

In [42]:
distances = []
for i in range(len(topic_representatives)):
   d = []
   for j in range(len(topic_representatives)):
      if i != j:
        d.append(1-spatial.distance.cosine(topic_representatives[i], topic_representatives[j]))
   distances.append(d)

Tramite la cella seguente andiamo a visualizzare le distanze semantiche individuate.

In [43]:
for i in range(len(distances)):
  for j in range(len(distances[i])):
    if j < i:
      print(f'Distanza semantica del topic {i} dal topic {j}: {distances[i][j]}')
    else:
      print(f'Distanza semantica del topic {i} dal topic {j+1}: {distances[i][j]}')
  print("------------------")

Distanza semantica del topic 0 dal topic 1: 0.570235550403595
Distanza semantica del topic 0 dal topic 2: 0.6763640642166138
Distanza semantica del topic 0 dal topic 3: 0.5634257197380066
Distanza semantica del topic 0 dal topic 4: 0.5360941290855408
Distanza semantica del topic 0 dal topic 5: 0.5423681735992432
Distanza semantica del topic 0 dal topic 6: 0.3063722550868988
Distanza semantica del topic 0 dal topic 7: 0.6138285994529724
------------------
Distanza semantica del topic 1 dal topic 0: 0.570235550403595
Distanza semantica del topic 1 dal topic 2: 0.5443044304847717
Distanza semantica del topic 1 dal topic 3: 0.7978641390800476
Distanza semantica del topic 1 dal topic 4: 0.5355232954025269
Distanza semantica del topic 1 dal topic 5: 0.5377824902534485
Distanza semantica del topic 1 dal topic 6: 0.24029801785945892
Distanza semantica del topic 1 dal topic 7: 0.3765726387500763
------------------
Distanza semantica del topic 2 dal topic 0: 0.6763640642166138
Distanza semantica

Come si può vedere dall'output della cella precedente, i topic sono ben distinti.

**Punto 4**

In [53]:
df_nospam = df[df['label_num'] == 0]

In [54]:
df_nospam.shape

(3672, 2)

In [55]:
nlp = spacy.load('en_core_web_sm')

In [58]:
organizations = []
for sentence in df_nospam['text']:
    doc = nlp(sentence)
    for token in doc:
        if str(token.ent_type_) == 'ORG':
          organizations.append(str(token))

In [60]:
org = set(organizations)

In [61]:
print(org)

{'meters', 'shoreline', 'watches', 'evans', 'veselack', 'cayanosa', 'pacific', 'kim', 'ps', 'sql', 'lrc', 'aburrell', 'shut', 'north', 'steven', 'michelle', 'krishnaswamy', 'guerra', 'other', 'chokshi', 'carrabine', 'western', 'now', 'robin', 'investments', 'brooks', 'baptist', 'pec', 'paso', 'lebroc', 'aquila', 'coleman', 'mundt', 'white', 'errigo', 'sigmund', 'phillips', 'staff', 'dropbox', 'home', 'net', 'merchant', 'ceo', 'cec', 'schield', 'teco', 'milnthorp', 'refinery', 'ga', 'falcone', 'for', 'landman', 'sap', 'anne', 'espinoza', 'federal', 'list', 'advisors', 'p', 'stephanie', 'co', 'portland', 'cornshucker', 'east', 'panther', 'wal', 'morris', 'gore', 'corey', 'srm', 'america', 'states', '7', 'marron', 'and', 'lloyds', 'stephenson', 'lagrasta', 'jayson', 'thanls', '77002', 'congress', 'koenig', 'hillman', 'telephone', 'hobbs', 'yuma', 'ferries', '679847', 'luu', 'county', 'clynes', 'under', 'byron', 'kelly', 'future', 'lia', 'phibro', 'lockhart', 'bruce', 'jprejean', 'kemp', '